In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv
import scipy.stats as stats

In [3]:
wine_data = "Datasets/winemag-data-130k-v2.csv"

In [4]:
wine_data_df = pd.read_csv(wine_data)

In [5]:
wine_data_df.head(2)

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos


In [6]:
wine_data_df.columns

Index(['Unnamed: 0', 'country', 'description', 'designation', 'points',
       'price', 'province', 'region_1', 'region_2', 'taster_name',
       'taster_twitter_handle', 'title', 'variety', 'winery'],
      dtype='object')

In [7]:
reduced_wine_data_df = wine_data_df[["country", "points", "price", "variety", "province", "winery"]]

In [8]:
drop_na_df = reduced_wine_data_df.dropna()

In [9]:
drop_na_df.count()

country     120915
points      120915
price       120915
variety     120915
province    120915
winery      120915
dtype: int64

In [10]:
#groupbys for dataset
country_group = drop_na_df.groupby("country")
province_group = drop_na_df.groupby("province")
winery_group = drop_na_df.groupby("winery")
variety_group = drop_na_df.groupby("variety")
country_by_province = drop_na_df.groupby(["country", "province"])
country_by_province.mean()
country_by_variety = drop_na_df.groupby(["country", "variety"])
country_group.mean()

,points,price
country,,
Argentina,86.710330,24.510117
Armenia,87.500000,14.500000
Australia,88.595466,35.437663
Austria,90.190782,30.762772
Bosnia and Herzegovina,86.500000,12.500000
Brazil,84.659574,23.765957
Bulgaria,87.936170,14.645390
Canada,89.377953,35.712598
Chile,86.495130,20.787316


In [11]:

most_produced_varieties= drop_na_df.groupby("variety").count()["points"].sort_values(ascending = False)
most_produced_varieties_df = pd.DataFrame({"Number of Bottles":most_produced_varieties})
top_10_varieties = most_produced_varieties_df.iloc[0:10, :].index.to_list()
top_10_varieties

['Pinot Noir',
 'Chardonnay',
 'Cabernet Sauvignon',
 'Red Blend',
 'Bordeaux-style Red Blend',
 'Riesling',
 'Sauvignon Blanc',
 'Syrah',
 'Rosé',
 'Merlot']

In [12]:
country_production = drop_na_df["country"].value_counts()

In [13]:
#Number of bottles with score greater than 80 indexed by country in descending order
sort_production_df = pd.DataFrame({"Bottles with Score >80":country_production})
#iloc df of top 10 producers for graphing
top_10_producers = sort_production_df.iloc[0:10,:]
top_10_producers

,Bottles with Score >80
US,54265
France,17776
Italy,16914
Spain,6573
Portugal,4875
Chile,4415
Argentina,3756
Austria,2799
Australia,2294
Germany,2120


In [14]:
#average score in descending order
avg_points_descending=country_group.mean()["points"].sort_values(ascending=False)
sort_scores = pd.DataFrame({"Average Score":avg_points_descending})
#iloc of top 10 scorers for graphing (convert to value_counts for series if needed)
top_10_scorers_df = sort_scores.iloc[0:10,:]
#list for loc searching of dataframes
top_10_scorers_list = top_10_scorers_df.index.to_list()


In [15]:
#Dataframe of top 10 producers average score (optional sort by descending)
top_10_producers_list = top_10_producers.index.to_list()
top_10_producers_scores = sort_scores.loc[top_10_producers_list,:]
top_10_producers_scores.sort_values(by = "Average Score", ascending = False)





,Average Score
country,
Austria,90.190782
Germany,89.836321
France,88.734867
Italy,88.618186
Australia,88.595466
US,88.566387
Portugal,88.316718
Spain,87.290735
Argentina,86.710330


In [16]:
#highest scoring varieties
mean_by_variety = variety_group.mean()
top_varieties = mean_by_variety.sort_values(by=["points"], ascending = False)
#most reviewed varieties
most_pop_varieties = variety_group.count().sort_values(by = ['points'], ascending = False)
top_10_varieties = most_pop_varieties.iloc[0:10,:].index.to_list()
#top 10 number of reviews by score and price
top_10_varieties_df = mean_by_variety.loc[top_10_varieties, :]
top_10_varieties_df.sort_values(by = ['points'], ascending = False)
top_10_producers

,Bottles with Score >80
US,54265
France,17776
Italy,16914
Spain,6573
Portugal,4875
Chile,4415
Argentina,3756
Austria,2799
Australia,2294
Germany,2120


In [17]:
select_countries = ['US','France', 'Italy', 'Spain', 'Portugal', 'Chile', 'Argentina', 'Austria', 'Australia', 'Germany']
select_df = drop_na_df[drop_na_df['country'].isin(select_countries)]

select_df.to_csv("Datasets/cleaned_countries.csv")


In [18]:
#Dataframes of top 10 varieties 
riesling_df = select_df[select_df['variety']=="Riesling"]
pinot_noir_df = select_df[select_df['variety']=="Pinot Noir"]
syrah_df = select_df[select_df['variety']=="Syrah"]
bordeaux_df = select_df[select_df['variety']=="Bordeaux-style Red Blend"]
cabernet_sauvignon_df = select_df[select_df['variety']=="Cabernet Sauvignon"]
red_blend_df = select_df[select_df['variety']=="Red Blend"]
chardonnay_df = select_df[select_df['variety']=="Chardonnay"]
sauvignon_blanc_df = select_df[select_df['variety']=="Sauvignon Blanc"]
rose_df = select_df[select_df['variety']=="Rosé"]
merlot_df = select_df[select_df['variety']=="Merlot"]
#I can set the indexes by country for these if we want to run some ttests,
#just be warned i was told that this was bad practice

In [19]:
#Mean groupbys of top 10 varieties by country
#These are means, not counts, and are necessarily means of score *and* price by country for varieties of wine
#You can get these values by grouping by country and variety, but these are simplified dataframes for easy graphing

riesling_mean = riesling_df.groupby("country").mean()
pinot_noir_mean = pinot_noir_df.groupby("country").mean()
syrah_mean = syrah_df.groupby("country").mean()
bordeaux_mean = bordeaux_df.groupby("country").mean()
cabernet_mean = cabernet_sauvignon_df.groupby("country").mean()
red_blend_mean = red_blend_df.groupby("country").mean()
chardonnay_mean = chardonnay_df.groupby("country").mean()
sauvignon_mean = sauvignon_blanc_df.groupby("country").mean()
rose_mean = rose_df.groupby("country").mean()
merlot_mean = merlot_df.groupby("country").mean()


In [21]:
us_df = select_df[select_df["country"]=="US"]
france_df = select_df[select_df["country"]=="France"]
italy_df = select_df[select_df["country"]=="Italy"]
spain_df = select_df[select_df["country"]=="Spain"]
portugal_df = select_df[select_df["country"]=="Portugal"]
chile_df = select_df[select_df["country"]=="Chile"]
argentina_df = select_df[select_df["country"]=="Argentina"]
austria_df = select_df[select_df["country"]=="Austria"]
australia_df = select_df[select_df["country"]=="Australia"]
germany_df = select_df[select_df["country"]=="Germany"]

In [26]:
ten_by_ten = pd.DataFrame({"Riesling" : riesling_mean["points"],
                           "Pinot Noir" : pinot_noir_mean["points"],
                           "Syrah": syrah_mean["points"],
                           "Bordeaux": bordeaux_mean["points"],
                           "Cabernet" : cabernet_mean["points"],
                           "Red Blend": red_blend_mean["points"],
                           "Chardonnay": chardonnay_mean["points"],
                           "Sauvignon Blanc": sauvignon_mean["points"],
                           "Rosé": rose_mean["points"],
                           "Merlot": merlot_mean["points"]})
ten_by_ten.

,Riesling,Pinot Noir,Syrah,Bordeaux,Cabernet,Red Blend,Chardonnay,Sauvignon Blanc,Rosé,Merlot
Argentina,86.000000,85.611650,85.785714,89.848837,86.055970,88.422907,84.941176,84.346154,84.081081,85.034483
Australia,88.979592,87.354331,91.625000,89.137931,89.228137,87.833333,87.339241,87.020833,87.250000,85.736842
Austria,91.495935,90.433735,89.000000,90.500000,87.400000,90.085366,90.185185,89.505376,88.017241,91.500000
Chile,86.000000,85.921788,88.000000,90.135135,86.680628,88.948020,85.141176,85.974815,84.232558,85.257812
France,90.658951,89.768401,89.875000,88.159471,85.433962,88.125926,89.301794,88.417476,87.291889,86.584906
Germany,90.080388,89.436364,NaN,NaN,NaN,86.750000,89.400000,NaN,89.000000,NaN
Italy,87.909091,86.833333,88.868217,89.500000,89.177966,88.851977,88.230769,86.844444,86.018519,89.594203
Portugal,87.666667,88.200000,88.769231,93.000000,88.400000,88.571429,87.647059,86.695652,84.602941,88.750000
Spain,82.000000,87.000000,88.470588,86.888889,87.211538,88.143564,84.816901,84.363636,84.851351,84.933333
US,87.958166,89.610125,89.432550,89.688498,89.030357,87.852782,88.446331,87.479109,87.226667,87.383913
